In [23]:
import os
from sklearn import metrics 
import pickle
import pandas as pd
import numpy as np

In [8]:
# Load Models
models = []
for f in os.listdir('../models/'):
    if f.endswith('.pkl'):
        with open('../models/' + f, 'rb') as f:
            models.append(pickle.load(f))

In [27]:
# Assemble df
model_name = lambda model: str(model.__class__).split('.')[-1][:-2]

names = [model_name(model) for model in models]
names[-3] += ' (Poly)'
names[-2] += ' (RBF)'
names

cols = ['auc', 'f1', 'precision', 'recall']

df = pd.DataFrame(data=np.zeros((len(names), len(cols))), columns=cols, index=names)
df

,auc,f1,precision,recall
XGBClassifier,0.0,0.0,0.0,0.0
LogisticRegression,0.0,0.0,0.0,0.0
GaussianNB,0.0,0.0,0.0,0.0
CatBoostClassifier,0.0,0.0,0.0,0.0
GradientBoostingClassifier,0.0,0.0,0.0,0.0
RandomForestClassifier,0.0,0.0,0.0,0.0
KNeighborsClassifier,0.0,0.0,0.0,0.0
SVC (Poly),0.0,0.0,0.0,0.0
SVC (RBF),0.0,0.0,0.0,0.0
LGBMClassifier,0.0,0.0,0.0,0.0


In [41]:
# get predictions
df_clean = pd.read_csv('../data/telco_churn_clean.csv')
df_numer = pd.read_csv('../data/telco_churn_numeric.csv')

target_cat = df_clean['Churn'].replace('Yes', 1).replace('No', 0)
target = df_numer['Churn'].replace('Yes', 1).replace('No', 0)

df_clean.drop(['customerID', 'Churn'], axis=1, inplace=True)
df_numer.drop(['customerID', 'Churn'], axis=1, inplace=True)

preds = {}
for model in models:
    name = model_name(model)
    print(name)
    if name != 'CatBoostClassifier':
        preds[name] = model.predict_proba(df_numer)
    else:
        preds[name] = model.predict_proba(df_clean)

XGBClassifier
LogisticRegression
GaussianNB
CatBoostClassifier
GradientBoostingClassifier
RandomForestClassifier
KNeighborsClassifier
SVC


AttributeError: predict_proba is not available when  probability=False

In [40]:
# score models
for model in models:
    name = model_name(model)
    if name != 'CatBoostClassifier':
        y_true = target
    else:
        y_true = target_cat
    auc = metrics.roc_auc_score(y_true, preds[name])
    f1 = metrics.f1_score(y_true, preds[name])
    precision = metrics.precision_score(y_true, preds[name])
    recall = metrics.recall_score(y_true, preds[name])
    df.at[name, 'auc'] = auc
    df.at[name, 'f1'] = f1
    df.at[name, 'precision'] = precision
    df.at[name, 'recall'] = recall
df

ValueError: bad input shape (7043, 2)

In [38]:
for pred in preds.values():
    print(len(pred))

7043
7043
7043
7043
7043
7043
7043
7043
7043


In [37]:
preds.value

{'XGBClassifier': array([1, 0, 0, ..., 0, 1, 0]),
 'LogisticRegression': array([1, 1, 1, ..., 1, 1, 1]),
 'GaussianNB': array([0, 0, 0, ..., 0, 0, 0]),
 'CatBoostClassifier': array([0., 0., 0., ..., 0., 0., 0.]),
 'GradientBoostingClassifier': array([0, 0, 0, ..., 0, 0, 0]),
 'RandomForestClassifier': array([0, 0, 0, ..., 0, 0, 0]),
 'KNeighborsClassifier': array([0, 0, 0, ..., 0, 0, 0]),
 'SVC': array([0, 0, 0, ..., 0, 0, 0]),
 'LGBMClassifier': array([0, 0, 0, ..., 0, 0, 0])}